# Entrenando y evaluando modelos

Este notebook es una adaptación del [original de *Aurélien Gerón*](https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb), de su libro: [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 3rd Edition. Aurélien Géron](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/)

## Definiendo el *pipeline* de preprocesamiento

In [1]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
cat_pipeline = make_pipeline( # Pipeline for categorical features
    SimpleImputer(strategy="most_frequent"), # Impute missing values with the most frequent value
    OneHotEncoder(handle_unknown="ignore")) # One-hot encode the categorical features

In [3]:
class ClusterSimilarity(BaseEstimator, TransformerMixin): # Custom transformer to compute similarity with cluster center
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma # RBF kernel bandwidth
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, n_init=10, 
                              random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1., random_state=42)

In [4]:
def column_ratio(X): # Custom transformer to compute the ratio of two columns
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in): # Custom function to name the output columns
    return ["ratio"]  # feature names out

def ratio_pipeline(): # Pipeline for ratio features (create new features by dividing two columns)
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler())

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler())

default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                     StandardScaler())
preprocessing = ColumnTransformer([
        ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]), # razón entre total_bedrooms y total_rooms (nueva feature)
        ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]), # razón entre total_rooms y households (nueva feature)
        ("people_per_house", ratio_pipeline(), ["population", "households"]), # razón entre population y households (nueva feature)
        ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                               "households", "median_income"]), # logaritmo de las columnas seleccionadas (para cambiar distribuciones sesgadas -skewed- por distribuciones normales)
        ("geo", cluster_simil, ["latitude", "longitude"]), # similitud con los clusters
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)), # pipeline categórico
    ],
    remainder=default_num_pipeline)  # one column remaining: housing_median_age

In [5]:
preprocessing

ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                             SimpleImputer(strategy='median')),
                                            ('standardscaler',
                                             StandardScaler())]),
                  transformers=[('bedrooms',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('functiontransformer',
                                                  FunctionTransformer(feature_names_out=<function ratio_name at 0x775ecef440e0>,
                                                                      func=<function column_ratio...
                                 ['total_bedrooms', 'total_rooms', 'population',
                                  'households', 'median_income']),
                                ('geo', ClusterSimilarity(random_state=42),
                                 ['latitude', 'longitude']),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x775ecef40230>)])

## Entrenando y evaluando modelos

### Importación y preparación de los datos

In [6]:
import pandas as pd
housing = pd.read_csv("./data/housing.csv")

In [7]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

strat_train_set, strat_test_set = train_test_split(
    housing, test_size=0.2, stratify=housing["income_cat"], random_state=42)
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
    
X_train = strat_train_set.drop("median_house_value", axis=1)
y_train = strat_train_set["median_house_value"].copy()

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()


### Definición de un pipeline completo con un modelo de regresión lineal

In [8]:
from sklearn.linear_model import LinearRegression

lin_reg = make_pipeline(preprocessing, LinearRegression())
lin_reg.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x775...
                                                   'median_income']),
                                                 ('geo',
                                                  ClusterSimilarity(random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x775ecef40230>)])),
                ('linearregression', LinearRegression())])

In [9]:
y_pred = lin_reg.predict(X_test) # Realizamos predicciones con los datos de test

Ahora podemos comparar algunos de los resultados predichos con sus etiquetas reales:

In [10]:
print("Valores reales:", list(y_test.iloc[:10]))
print("Predicciones:", list(y_pred[:10].round(-2)))

Valores reales: [500001.0, 162500.0, 204600.0, 159700.0, 184000.0, 151900.0, 104900.0, 500001.0, 367400.0, 346500.0]
Predicciones: [392900.0, 244200.0, 278500.0, 210900.0, 264800.0, 217300.0, 150300.0, 380500.0, 311900.0, 267000.0]


y ver el error porcentual en estas predicciones:

In [11]:
error_ratios = y_pred.round(-2) / y_test - 1
print(", ".join([f"{100 * ratio:.1f}%" for ratio in error_ratios]))

-21.4%, 50.3%, 36.1%, 32.1%, 43.9%, 43.1%, 43.3%, -23.9%, -15.1%, -22.9%, -7.8%, -14.5%, 15.6%, 8.2%, 29.3%, 45.2%, -1.5%, 122.0%, -12.2%, -8.8%, 20.0%, -7.2%, -21.1%, 71.9%, 71.1%, -19.0%, 8.3%, -3.1%, 35.4%, 11.3%, -28.1%, -95.8%, 2.7%, -35.8%, -36.5%, -14.7%, 21.4%, 18.1%, 19.7%, -26.8%, 28.7%, -11.5%, 20.4%, 23.8%, 17.2%, 38.4%, 100.8%, 36.2%, -7.9%, -30.5%, -4.5%, -0.9%, 13.9%, -10.7%, -12.0%, 26.7%, -4.8%, 2.4%, -30.9%, -8.6%, 1.5%, -4.7%, 1.3%, -8.8%, -2.6%, -8.6%, -1.8%, -24.5%, 50.0%, 0.5%, -12.1%, 11.0%, 43.1%, 7.4%, -11.3%, 13.4%, -28.8%, -15.6%, 5.6%, -64.0%, 31.6%, 4.3%, -0.2%, -22.4%, 0.2%, -40.5%, 11.9%, 76.8%, -32.0%, 25.2%, 12.1%, -13.8%, 52.6%, 81.5%, 41.2%, 51.9%, 39.3%, 49.8%, 42.3%, -36.9%, 31.0%, -27.7%, 47.5%, 149.8%, 7.2%, -17.5%, 30.7%, -24.7%, 37.3%, 1.7%, -77.0%, -2.7%, -10.6%, 78.2%, -5.3%, -89.8%, -17.2%, 2.5%, -18.9%, 6.0%, 3.2%, -9.9%, -22.5%, 21.2%, -29.5%, -15.5%, 25.8%, 52.9%, 38.9%, 56.3%, -4.7%, 0.1%, 14.7%, -7.1%, -128.8%, -30.5%, -57.7%, 34.1%, 51.

pero podemos ver el rendimiento con el error cuadrático medio, como habíamos estipulado:

In [12]:
from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(y_test, y_pred)

68812.13500088906

Un error de 68812$ para predicciones del valor de viviendas cuyo precio medio es de 206856$ no parece muy útil.

### Probando otro modelo: Árbol de decisión

In [13]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x775...
                                                  ClusterSimilarity(random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x775ecef40230>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [14]:
y_pred = tree_reg.predict(X_test)
root_mean_squared_error(y_test, y_pred)

66946.50354073904

Vemos que el error medido en el conjunto de test sigue siendo demasiado alto para considerar que tenemos un modelo útil (una estimación de precio de vivienda con un error de 68000$ es probablemente mucho peor que la que haría subjetivamente un profesional del sector).

### *Data Leakage*

Sin embargo, acabamos de incurrir en un error común conocido como ***data leakage*** o fuga de datos: el error en el conjunto de test no se debe usar para comparar modelos o para ajustar hiperparámetros, por los siguientes motivos:

- Contaminación de datos: Si ajustas continuamente tu modelo basándote en el error del conjunto de test, este empieza a influir en las decisiones que tomas. Indirectamente, el modelo comienza a "aprender" los datos del test, lo que genera un problema conocido como sobreajuste al conjunto de test.

- Evaluación sesgada: El conjunto de test pierde su rol como indicador objetivo de generalización. Esto implica que ya no podemos confiar en el error del test como una estimación imparcial del rendimiento en datos completamente nuevos.

- Necesidad de un conjunto no contaminado: El conjunto de test debe ser utilizado solo una vez, al final, para evaluar el modelo final. Esto garantiza que sea un reflejo real de cómo el modelo se comportará en datos no vistos.

Si ajustas modelos basándote en el error de test, puedes obtener un modelo que parece bueno en el conjunto de test, pero que tiene un rendimiento pobre en datos completamente nuevos.


## Conjunto de validación


Para evitar usar el conjunto de test indebidamente durante el desarrollo del modelo, normalmente se divide el conjunto de datos en tres partes:

- **Entrenamiento**: Para ajustar los **parámetros** del modelo.
- **Validación**: Para ajustar **hiperparámetros** y **comparar diferentes modelos**. Este conjunto permite medir el rendimiento intermedio sin comprometer el conjunto de test.
- **Test**: Para evaluar el modelo final una sola vez. Debe permanecer intacto durante todo el proceso de desarrollo.

El conjunto de validación se utiliza para comparar modelos y los ajustes de hiperparámetros. El conjunto de test se guarda para evaluar el rendimiento final del modelo elegido que se va a utilizar en producción.

## *Cross-validation*

Hasta ahora hemos separado en dos conjuntos el de entrenamiento y el de test. Sin embargo, en muchos casos, el rendimiento variará dependiendo del muestreo que hayamos hecho. Si dejamos de fijar la semilla de muestreo (parámetro `random_state` de la función `train_test_split`), obtendremos resultados distintos (si bien en este caso no varían demasiado en ninguno de los dos modelos).

Una forma más eficiente de hacerlo es la **validación cruzada o *cross-validation***: en lugar de dividir el conjunto de entrenamiento en dos, se divide en *k* conjuntos (*folds*). Luego se entrena el modelo *k* veces, cada vez dejando un conjunto distinto como conjunto de validación y los otros *k-1* como conjunto de entrenamiento. El resultado es un array con *k* puntuaciones. 

Por ejemplo, con el siguiente código realiza el entrenamiento con 10 muestreos distintos. Los resultados serán similares a los que podríamos obtener ejecutando 10 veces el código sin fijar la semilla de muestreo aleatorio. La contrapartida es evidente: el coste computacional también se multiplica por 10.

Aparece así el concepto de **conjunto de validación**. El conjunto de validación se utiliza para comparar modelos y los **ajustes de hiperparámetros**, y es el que va cambiando en cada iteración e la validación cruzada. El conjunto de test se guarda para evaluar el rendimiento final del modelo elegido una vez que se ha entrenado.

[<img src="./img/cross-validation.png" width="500">](https://www.researchgate.net/figure/Train-test-cross-validation-split-methodology-used-in-this-paper-The-first-operation_fig2_340567535)

[<img src="./img/cross-validation2.png" width="700">](https://www.statology.org/validation-set-vs-test-set/)

In [15]:
from sklearn.model_selection import cross_val_score

tree_rmses = -cross_val_score(estimator = tree_reg, 
                              X = X_train,
                              y = y_train,
                              scoring = "neg_root_mean_squared_error",
                              cv = 10) # 10-fold cross-validation

print(tree_rmses)
pd.Series(tree_rmses).describe()

[67450.10319744 68202.41365325 62118.73084295 64174.53169307
 63787.17957149 69273.33071392 69091.09129775 70659.65916181
 65425.33990029 74127.49155449]


count       10.000000
mean     67430.987159
std       3623.038287
min      62118.730843
25%      64487.233745
50%      67826.258425
75%      69227.770860
max      74127.491554
dtype: float64

El parámetro `scoring` de la función `cross_val_score` espera una **función de utilidad** (mayor es mejor) en lugar de una **función de coste** (menor es mejor), por lo que la puntuación es en realidad el opuesto del RMSE. Es un valor negativo, por lo que necesitamos cambiar el signo de la salida para obtener los valores de RMSE. `cross_val_score` buscará maximizar el *score*, de modo que al maximizar el negativo del RMSE, se minimiza el RMSE.

Existen multiples *strings* idenficativos de las métricas de evaluación que se pueden utilizar en `scoring` y que se pueden consultar en la [documentación de Scikit-Learn](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).

Podemos ver que recuperamos un RMSE medio de 67431$ **en validación** con una desviación estándar de 3623$. Nos aporta información más detallada (y menos dependiente del muestreo que hayamos hecho) sobre el rendimiento.

Ahora tenemos una métrica de evaluación de nuestro modelo con árbol de decisión que podemos comparar con otros sin tocar el conjunto de test.

## Comparando otro modelo (*Random Forest*)

*Randon Forest* es un modelo que entrena varios árboles de decisión (***ensemble learning***) en subconjuntos aleatorios de las características y promedia sus predicciones.

In [16]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))
forest_rmses = -cross_val_score(forest_reg, X_train, y_train, scoring = "neg_root_mean_squared_error", cv = 10)
pd.Series(forest_rmses).describe()

count       10.000000
mean     47328.782275
std       2456.436091
min      43953.861521
25%      45272.501522
50%      47431.774424
75%      49045.891146
max      51153.378784
dtype: float64

Random Forest consigue una mejoría (47328$ de error medio) en el conjunto de validación respecto al árbol de decisión simple.

Aunque sigue siendo un error alto es el mejor modelo que tenemos hasta ahora. Asumiendo que nos quedásemos con él, podríamos finalmente entrenar el modelo elegido con el conjunto de entrenamiento entero y ver su rendimiento con el conjunto de test que mantuvimos separado antes de ponerlo en producción.

In [17]:
y_pred = forest_reg.fit(X_train, y_train).predict(X_test)
forest_rmse = root_mean_squared_error(y_test, y_pred)
forest_rmse

45682.20027231889